# Importando módulos 

In [176]:
import obspy
from obspy.taup import TauPyModel

from multiprocessing import Pool
from obspy import read,UTCDateTime,Trace
from obspy.clients.fdsn import Client
import os
import glob
import numpy as np
from collections import defaultdict
import pandas as pd

#para plotar as figuras
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition,inset_axes
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

from datetime import datetime,timedelta,date
from tqdm import tqdm

from shapely.geometry.polygon import LinearRing

import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from IPython.display import HTML
from IPython import display

# Inputs e Outputs

In [177]:
FOLDER_OUTPUT = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/'

MSEED_INPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/MSEED/"

METADATA_OUTPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/METADATA/"

filename_csv = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/metadados_glider_acustico_pmpas-bs.csv'

campanha_csv = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/Dados_campanhas_Gliders.xls'

# Extraindo informações dos arquivos ".mseed"

In [10]:
filenames_MSEED = sorted(glob.glob(MSEED_INPUT+'*/*/*.mseed'))

In [11]:
def mseed_data_2_dataframe(i):
    subdir, filename_wav = os.path.split(i)
    filename = filename_wav.split('.mseed')[0]
    if 'pa' in filename.split('_')[0]:
        mergulho = filename.split('_')[0].split('a')[1]
        stream_number = filename.split('_')[1]

        year_month_day = filename.split('_')[2]
        hour_minute_second = filename.split('_')[3]

        year = int('20'+year_month_day[:2])
        month = int(year_month_day[2:4])
        day = int(year_month_day[4:])

        hour = int(hour_minute_second[:2])
        minute = int(hour_minute_second[2:4])
        second = int(hour_minute_second[4:])

        d = UTCDateTime(datetime(year,month,day,hour,minute,second).isoformat())


    if 'pa' in filename.split('_')[2]:

        mergulho = filename.split('_')[2].split('a')[1]
        stream_number = filename.split('_')[3]

        year_month_day = filename.split('_')[0]
        hour_minute_second = filename.split('_')[1]

        year = int('20'+year_month_day[:2])
        month = int(year_month_day[2:4])
        day = int(year_month_day[4:])

        hour = int(hour_minute_second[:2])
        minute = int(hour_minute_second[2:4])
        second = int(hour_minute_second[4:])

        d = UTCDateTime(datetime(year,month,day,hour,minute,second).isoformat())
        
    
    st = read(i,headonly=True)   
    #----------------------------
    #Starting Dataframe

    starttime = st[0].stats.starttime.datetime
    endtime = st[0].stats.endtime.datetime
    sampling_rate = st[0].stats.sampling_rate
    npts = st[0].stats.npts

    
    df = pd.DataFrame([[filename],[mergulho],[stream_number],[starttime],[endtime],[sampling_rate],[npts]], index=['filename_mseed', 'mergulho', 'stream_number','starttime','endtime','sampling_rate','npts']).T
    
    #Ending Dataframe
    #----------------------------
    return df

In [12]:
pandas_mseed_lst = []

with Pool(processes=4) as p:
    max_ = len(filenames_MSEED)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(mseed_data_2_dataframe,filenames_MSEED):
            pbar.update()
            pandas_mseed_lst.append(result)

100%|██████████████████████████████████| 124257/124257 [11:23<00:00, 181.70it/s]


In [178]:
dataframe_mseed_final = pd.concat(pandas_mseed_lst, ignore_index=True)

In [179]:
dataframe_mseed_final['starttime'] = pd.to_datetime(dataframe_mseed_final.starttime, format='%Y-%m-%d %H:%M:%S')
dataframe_mseed_final['endtime'] = pd.to_datetime(dataframe_mseed_final.endtime, format='%Y-%m-%d %H:%M:%S')


In [180]:
dataframe_mseed_final.sort_values(by='starttime')

,filename_mseed,mergulho,stream_number,starttime,endtime,sampling_rate,npts
0,pa0001au_001_151112_134841,0001,001,2015-11-12 13:48:41,2015-11-12 13:50:40.930,100.0,11994
3,pa0001au_002_151112_135041,0001,002,2015-11-12 13:50:41,2015-11-12 13:52:40.930,100.0,11994
2,pa0001au_003_151112_135241,0001,003,2015-11-12 13:52:41,2015-11-12 13:54:40.930,100.0,11994
1,pa0001au_004_151112_135441,0001,004,2015-11-12 13:54:41,2015-11-12 13:56:40.930,100.0,11994
4,pa0001au_005_151112_135641,0001,005,2015-11-12 13:56:41,2015-11-12 13:57:00.980,100.0,1999
...,...,...,...,...,...,...,...
124251,pa0296au_008_211104_061338,0296,008,2021-11-04 06:13:38,2021-11-04 06:23:37.990,100.0,60000
124252,pa0296au_009_211104_062338,0296,009,2021-11-04 06:23:38,2021-11-04 06:33:37.990,100.0,60000
124253,pa0296au_010_211104_063338,0296,010,2021-11-04 06:33:38,2021-11-04 06:43:37.990,100.0,60000
124255,pa0296au_011_211104_064338,0296,011,2021-11-04 06:43:38,2021-11-04 06:53:37.990,100.0,60000


In [181]:
dataframe_mseed_final['date'] = dataframe_mseed_final['starttime'].dt.date

In [182]:
dataframe_mseed_final

,filename_mseed,mergulho,stream_number,starttime,endtime,sampling_rate,npts,date
0,pa0001au_001_151112_134841,0001,001,2015-11-12 13:48:41,2015-11-12 13:50:40.930,100.0,11994,2015-11-12
1,pa0001au_004_151112_135441,0001,004,2015-11-12 13:54:41,2015-11-12 13:56:40.930,100.0,11994,2015-11-12
2,pa0001au_003_151112_135241,0001,003,2015-11-12 13:52:41,2015-11-12 13:54:40.930,100.0,11994,2015-11-12
3,pa0001au_002_151112_135041,0001,002,2015-11-12 13:50:41,2015-11-12 13:52:40.930,100.0,11994,2015-11-12
4,pa0001au_005_151112_135641,0001,005,2015-11-12 13:56:41,2015-11-12 13:57:00.980,100.0,1999,2015-11-12
...,...,...,...,...,...,...,...,...
124252,pa0296au_009_211104_062338,0296,009,2021-11-04 06:23:38,2021-11-04 06:33:37.990,100.0,60000,2021-11-04
124253,pa0296au_010_211104_063338,0296,010,2021-11-04 06:33:38,2021-11-04 06:43:37.990,100.0,60000,2021-11-04
124254,pa0296au_012_211104_065338,0296,012,2021-11-04 06:53:38,2021-11-04 06:57:21.780,100.0,22379,2021-11-04
124255,pa0296au_011_211104_064338,0296,011,2021-11-04 06:43:38,2021-11-04 06:53:37.990,100.0,60000,2021-11-04


# Extraindo informações do arquivo ".csv" com as campanhas

In [117]:
ciclos_dataframe_csv = pd.read_excel(campanha_csv)
ciclos_dataframe_csv

,Sistema,Ciclo,starttime,endtime,Fs,nBits,Hidrofone,PreAmp,N/S
0,GLIDER,1,2015-11-10,2015-12-20,125000,16,911006,1501,SG618
1,GLIDER,2,2016-01-08,2016-02-02,125000,16,911006,1501,SG618
2,GLIDER,3,2016-02-02,2016-04-20,125000,24,911008,1503,SG612
3,GLIDER,4,2016-04-20,2016-07-07,125000,24,911006,1501,SG612
4,GLIDER,5,2016-07-07,2016-08-16,125000,24,911008,1503,SG612
...,...,...,...,...,...,...,...,...,...
89,GLIDER,MC_C026,2023-06-22,2023-07-30,128000,24,G003654,AMAR-G4,SL976
90,GLIDER,ME_C012,2023-06-20,2023-08-01,128000,24,C000811,AMAR-G4,SL977
91,GLIDER,MC_C027,2023-12-30,2023-12-30,128000,24,G003654,AMAR-G4,SL975
92,GLIDER,MC_C028,2023-08-03,2023-09-09,128000,24,F001108,AMAR-G4,SL977


In [118]:
ciclos_dataframe_csv['starttime'] = pd.to_datetime(ciclos_dataframe_csv.starttime, format='%Y-%m-%d %H:%M:%S').dt.date
ciclos_dataframe_csv['endtime'] = pd.to_datetime(ciclos_dataframe_csv.endtime, format='%Y-%m-%d %H:%M:%S').dt.date

In [ ]:
ciclos_dataframe_csv['Ciclo_new'] = range(len(ciclos_dataframe_csv))

In [144]:
ciclos_dataframe_csv

,Sistema,Ciclo,starttime,endtime,Fs,nBits,Hidrofone,PreAmp,N/S,Ciclo_new
0,GLIDER,1,2015-11-10,2015-12-20,125000,16,911006,1501,SG618,0
1,GLIDER,2,2016-01-08,2016-02-02,125000,16,911006,1501,SG618,1
2,GLIDER,3,2016-02-02,2016-04-20,125000,24,911008,1503,SG612,2
3,GLIDER,4,2016-04-20,2016-07-07,125000,24,911006,1501,SG612,3
4,GLIDER,5,2016-07-07,2016-08-16,125000,24,911008,1503,SG612,4
...,...,...,...,...,...,...,...,...,...,...
89,GLIDER,MC_C026,2023-06-22,2023-07-30,128000,24,G003654,AMAR-G4,SL976,89
90,GLIDER,ME_C012,2023-06-20,2023-08-01,128000,24,C000811,AMAR-G4,SL977,90
91,GLIDER,MC_C027,2023-12-30,2023-12-30,128000,24,G003654,AMAR-G4,SL975,91
92,GLIDER,MC_C028,2023-08-03,2023-09-09,128000,24,F001108,AMAR-G4,SL977,92


# Extraindo informações do arquivo ".csv" com as localizações (lat,lon,dep)

In [183]:
dataframe_csv = pd.read_csv(filename_csv,parse_dates=['time'])
dataframe_csv.sort_values(by='time')

,time,filename,latitude,longitude,depth
0,2015-11-12 13:48:40.999995,pa0001au_001_151112_134841_rms_spl3.mat,-23.825157,-42.495483,1.314286
1,2015-11-12 13:49:41.000001,pa0001au_001_151112_134841_rms_spl3.mat,-23.824950,-42.495487,7.415555
2,2015-11-12 13:50:40.999998,pa0001au_002_151112_135041_rms_spl3.mat,-23.824745,-42.495490,15.776000
3,2015-11-12 13:51:41.000004,pa0001au_002_151112_135041_rms_spl3.mat,-23.824540,-42.495495,24.372000
4,2015-11-12 13:52:41.000001,pa0001au_003_151112_135241_rms_spl3.mat,-23.824331,-42.495500,32.912000
...,...,...,...,...,...
547748,2021-11-04 06:52:38.000000,pa0296au_011_211104_064338_rms_spl3.mat,-24.461742,-42.471264,866.012900
547749,2021-11-04 06:53:37.999997,pa0296au_012_211104_065338_rms_spl3.mat,-24.461819,-42.471302,873.586100
547750,2021-11-04 06:54:38.000003,pa0296au_012_211104_065338_rms_spl3.mat,-24.461980,-42.471510,879.579600
547751,2021-11-04 06:55:38.000000,pa0296au_012_211104_065338_rms_spl3.mat,-24.462152,-42.471740,884.380700


In [184]:
df_csv = dataframe_csv.groupby("filename").agg(pd.Series.tolist)

In [185]:
df_csv['filename_mseed'] = df_csv.index.str.replace('_rms_spl3.mat', '')

In [186]:
df_csv.sort_values(by='time')

,time,latitude,longitude,depth,filename_mseed
filename,,,,,
pa0001au_001_151112_134841_rms_spl3.mat,"[2015-11-12 13:48:40.999995, 2015-11-12 13:49:...","[-23.825157, -23.82495]","[-42.495483, -42.495487]","[1.3142858, 7.4155555]",pa0001au_001_151112_134841
pa0001au_002_151112_135041_rms_spl3.mat,"[2015-11-12 13:50:40.999998, 2015-11-12 13:51:...","[-23.824745, -23.82454]","[-42.49549, -42.495495]","[15.776, 24.372]",pa0001au_002_151112_135041
pa0001au_003_151112_135241_rms_spl3.mat,"[2015-11-12 13:52:41.000001, 2015-11-12 13:53:...","[-23.824331, -23.824125]","[-42.4955, -42.495502]","[32.912, 41.34]",pa0001au_003_151112_135241
pa0001au_004_151112_135441_rms_spl3.mat,"[2015-11-12 13:54:41.000004, 2015-11-12 13:55:41]","[-23.82392, -23.823713]","[-42.495506, -42.495506]","[48.91, 52.824]",pa0001au_004_151112_135441
pa0002au_001_151112_143618_rms_spl3.mat,"[2015-11-12 14:36:17.999998, 2015-11-12 14:37:...","[-23.817575, -23.81744]","[-42.49506, -42.494965]","[1.4671428, 9.36]",pa0002au_001_151112_143618
...,...,...,...,...,...
pa0296au_008_211104_061338_rms_spl3.mat,"[2021-11-04 06:13:37.999999, 2021-11-04 06:14:...","[-24.457632, -24.457657, -24.457726, -24.45786...","[-42.470257, -42.470303, -42.470325, -42.47033...","[550.4096, 559.3073, 567.8462, 575.10706, 582....",pa0296au_008_211104_061338
pa0296au_009_211104_062338_rms_spl3.mat,"[2021-11-04 06:23:38.000003, 2021-11-04 06:24:...","[-24.458628, -24.458672, -24.45872, -24.458822...","[-42.47047, -42.470528, -42.470562, -42.470573...","[629.96436, 637.68353, 646.56323, 654.6224, 66...",pa0296au_009_211104_062338
pa0296au_010_211104_063338_rms_spl3.mat,"[2021-11-04 06:33:37.999998, 2021-11-04 06:34:...","[-24.459415, -24.459513, -24.45957, -24.45963,...","[-42.4708, -42.470802, -42.470856, -42.4709, -...","[706.7263, 715.5506, 722.5956, 730.11316, 737....",pa0296au_010_211104_063338


In [188]:
def name_to_mergulho_stream(filename):
    if 'pa' in filename.split('_')[0]:
        mergulho = filename.split('_')[0].split('a')[1]

    if 'pa' in filename.split('_')[2]:
        mergulho = filename.split('_')[2].split('a')[1]
    
    return mergulho

In [189]:
def calcular_media(lista):
    if len(lista) == 0:
        return None  # Retorna None se a lista estiver vazia para evitar erros
    else:
        return pd.Series(lista).mean()

In [190]:
def flatten(l):
    return l[0]

In [191]:
df_csv['mergulho'] = df_csv['filename_mseed'].apply(name_to_mergulho_stream)

In [192]:
df_csv['date'] = df_csv['time'].apply(calcular_media).dt.date

In [193]:
df_csv

,time,latitude,longitude,depth,filename_mseed,mergulho,date
filename,,,,,,,
160708_172008_pa0003au_001_rms_spl3.mat,"[2016-07-08 17:20:07.999999, 2016-07-08 17:21:...","[-24.391026, -24.391314, -24.391335]","[-43.821663, -43.82173, -43.82174]","[0.9987981, 9.891874, 10.969999]",160708_172008_pa0003au_001,0003,2016-07-08
160708_172307_pa0003au_002_rms_spl3.mat,"[2016-07-08 17:23:07.000004, 2016-07-08 17:24:...","[-24.391356, -24.3916, -24.391617]","[-43.821743, -43.8218, -43.821804]","[12.030156, 23.590548, 24.420235]",160708_172307_pa0003au_002,0003,2016-07-08
160708_172607_pa0003au_003_rms_spl3.mat,"[2016-07-08 17:26:07.000003, 2016-07-08 17:27:...","[-24.391909, -24.39193, -24.392172]","[-43.821873, -43.82188, -43.821938]","[36.456642, 37.333202, 47.245785]",160708_172607_pa0003au_003,0003,2016-07-08
160708_172906_pa0003au_004_rms_spl3.mat,"[2016-07-08 17:29:05.999998, 2016-07-08 17:30:...","[-24.392193, -24.392475, -24.392496]","[-43.82194, -43.822006, -43.82201]","[48.01094, 58.52969, 59.289062]",160708_172906_pa0003au_004,0003,2016-07-08
160708_173205_pa0003au_005_rms_spl3.mat,"[2016-07-08 17:32:05.000003, 2016-07-08 17:33:...","[-24.392744, -24.392765, -24.393057]","[-43.822067, -43.822075, -43.822144]","[68.57156, 69.49031, 80.89149]",160708_173205_pa0003au_005,0003,2016-07-08
...,...,...,...,...,...,...,...
pa0626au_041_160418_075202_rms_spl3.mat,"[2016-04-18 07:52:02.000001, 2016-04-18 07:53:...","[-24.46632, -24.466305]","[-43.148865, -43.14886]","[897.4117, 899.9383]",pa0626au_041_160418_075202,0626,2016-04-18
pa0626au_042_160418_075402_rms_spl3.mat,"[2016-04-18 07:54:02.000004, 2016-04-18 07:55:02]","[-24.466183, -24.466167]","[-43.14885, -43.148846]","[918.85736, 921.4589]",pa0626au_042_160418_075402,0626,2016-04-18
pa0626au_043_160418_075602_rms_spl3.mat,"[2016-04-18 07:56:01.999997, 2016-04-18 07:57:...","[-24.466047, -24.466032]","[-43.148834, -43.14883]","[940.0661, 942.0864]",pa0626au_043_160418_075602,0626,2016-04-18


In [194]:
df_csv['lat'] =  df_csv['latitude'].apply(calcular_media)
df_csv['lon'] =  df_csv['longitude'].apply(calcular_media)
df_csv['dep'] =  df_csv['depth'].apply(calcular_media)
df_csv['time'] =  df_csv['time'].apply(flatten)

In [195]:
df_csv_mean = df_csv.drop(columns=['latitude','longitude','depth',])

In [197]:
df_csv_mean.sort_values(by='date')

,time,filename_mseed,mergulho,date,lat,lon,dep
filename,,,,,,,
pa0005au_015_151112_211136_rms_spl3.mat,2015-11-12 21:11:35.999997,pa0005au_015_151112_211136,0005,2015-11-12,-23.785293,-42.469156,258.771985
pa0004au_011_151112_180207_rms_spl3.mat,2015-11-12 18:02:06.999995,pa0004au_011_151112_180207,0004,2015-11-12,-23.791018,-42.473718,182.740000
pa0004au_012_151112_180407_rms_spl3.mat,2015-11-12 18:04:06.999998,pa0004au_012_151112_180407,0004,2015-11-12,-23.791007,-42.473650,199.017500
pa0006au_001_151112_235913_rms_spl3.mat,2015-11-12 23:59:13.000003,pa0006au_001_151112_235913,0006,2015-11-12,-23.780344,-42.469132,8.517000
pa0004au_013_151112_180607_rms_spl3.mat,2015-11-12 18:06:07.000001,pa0004au_013_151112_180607,0004,2015-11-12,-23.790993,-42.473582,215.688000
...,...,...,...,...,...,...,...
pa0295au_010_211104_015556_rms_spl3.mat,2021-11-04 01:55:55.999999,pa0295au_010_211104_015556,0295,2021-11-04,-24.440742,-42.465287,736.252016
pa0296au_009_211104_062338_rms_spl3.mat,2021-11-04 06:23:38.000003,pa0296au_009_211104_062338,0296,2021-11-04,-24.458968,-42.470625,664.644524
pa0295au_009_211104_014556_rms_spl3.mat,2021-11-04 01:45:56.000005,pa0295au_009_211104_014556,0295,2021-11-04,-24.439859,-42.465195,652.485512


In [235]:
df_csv_meanS = df_csv_mean.sort_values(by='time')

In [213]:
len(sorted(df_csv_mean['filename_mseed'].to_list())) - len(sorted(dataframe_mseed_final['filename_mseed'].to_list()))

-936

In [215]:
no_lst = []
for i in tqdm(dataframe_mseed_final['filename_mseed'].to_list()):
    if i not in df_csv_mean['filename_mseed'].to_list():
        no_lst.append(i)


100%|██████████████████████████████████| 124257/124257 [04:51<00:00, 426.15it/s]


# Buscando o nome dos gliders no dataframe das campanhas:

In [161]:
ciclos_dataframe_csv

,Sistema,Ciclo,starttime,endtime,Fs,nBits,Hidrofone,PreAmp,N/S,Ciclo_new
0,GLIDER,1,2015-11-10,2015-12-20,125000,16,911006,1501,SG618,0
1,GLIDER,2,2016-01-08,2016-02-02,125000,16,911006,1501,SG618,1
2,GLIDER,3,2016-02-02,2016-04-20,125000,24,911008,1503,SG612,2
3,GLIDER,4,2016-04-20,2016-07-07,125000,24,911006,1501,SG612,3
4,GLIDER,5,2016-07-07,2016-08-16,125000,24,911008,1503,SG612,4
...,...,...,...,...,...,...,...,...,...,...
89,GLIDER,MC_C026,2023-06-22,2023-07-30,128000,24,G003654,AMAR-G4,SL976,89
90,GLIDER,ME_C012,2023-06-20,2023-08-01,128000,24,C000811,AMAR-G4,SL977,90
91,GLIDER,MC_C027,2023-12-30,2023-12-30,128000,24,G003654,AMAR-G4,SL975,91
92,GLIDER,MC_C028,2023-08-03,2023-09-09,128000,24,F001108,AMAR-G4,SL977,92


In [145]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [229]:
def convert_36(line):
    '''
    Creating the station name according to the hexatrigesimal two-digit algoritm conversion
    '''
    cycle_lst = [i for i in range(500)]
    
    # hexatrigesimal two digit number for each cycle:
    hexatri_lst = [str(np.base_repr(y, 36)).zfill(2) for y in cycle_lst]
    
    #find the nearest date and append the 2-hexatrigesimal-digit:
    return 'G'+line['starttime'].strftime('%y')+hexatri_lst[cycle_lst.index(nearest(cycle_lst,line['Ciclo_new']))]

In [257]:
ciclos_dataframe_csvS = ciclos_dataframe_csv.sort_values(by='starttime')

In [258]:
ciclos_dataframe_csvS

,Sistema,Ciclo,starttime,endtime,Fs,nBits,Hidrofone,PreAmp,N/S,Ciclo_new
0,GLIDER,1,2015-11-10,2015-12-20,125000,16,911006,1501,SG618,0
1,GLIDER,2,2016-01-08,2016-02-02,125000,16,911006,1501,SG618,1
2,GLIDER,3,2016-02-02,2016-04-20,125000,24,911008,1503,SG612,2
3,GLIDER,4,2016-04-20,2016-07-07,125000,24,911006,1501,SG612,3
4,GLIDER,5,2016-07-07,2016-08-16,125000,24,911008,1503,SG612,4
...,...,...,...,...,...,...,...,...,...,...
88,GLIDER,MC_C025,2023-06-21,2023-07-29,128000,24,F001108,AMAR-G4,SL975,88
89,GLIDER,MC_C026,2023-06-22,2023-07-30,128000,24,G003654,AMAR-G4,SL976,89
92,GLIDER,MC_C028,2023-08-03,2023-09-09,128000,24,F001108,AMAR-G4,SL977,92
93,GLIDER,MC_C029,2023-09-24,2023-10-05,128000,24,I000091,AMAR-G4,SL1099,93


In [259]:
ciclos_dataframe_csvS['station_nettab'] = ciclos_dataframe_csvS.apply(convert_36,axis=1)

In [260]:
ciclos_dataframe_csvS

,Sistema,Ciclo,starttime,endtime,Fs,nBits,Hidrofone,PreAmp,N/S,Ciclo_new,station_nettab
0,GLIDER,1,2015-11-10,2015-12-20,125000,16,911006,1501,SG618,0,G1500
1,GLIDER,2,2016-01-08,2016-02-02,125000,16,911006,1501,SG618,1,G1601
2,GLIDER,3,2016-02-02,2016-04-20,125000,24,911008,1503,SG612,2,G1602
3,GLIDER,4,2016-04-20,2016-07-07,125000,24,911006,1501,SG612,3,G1603
4,GLIDER,5,2016-07-07,2016-08-16,125000,24,911008,1503,SG612,4,G1604
...,...,...,...,...,...,...,...,...,...,...,...
88,GLIDER,MC_C025,2023-06-21,2023-07-29,128000,24,F001108,AMAR-G4,SL975,88,G232G
89,GLIDER,MC_C026,2023-06-22,2023-07-30,128000,24,G003654,AMAR-G4,SL976,89,G232H
92,GLIDER,MC_C028,2023-08-03,2023-09-09,128000,24,F001108,AMAR-G4,SL977,92,G232K
93,GLIDER,MC_C029,2023-09-24,2023-10-05,128000,24,I000091,AMAR-G4,SL1099,93,G232L


In [263]:
df_csv_meanS

,time,filename_mseed,mergulho,date,lat,lon,dep
filename,,,,,,,
pa0001au_001_151112_134841_rms_spl3.mat,2015-11-12 13:48:40.999995,pa0001au_001_151112_134841,0001,2015-11-12,-23.825054,-42.495485,4.364921
pa0001au_002_151112_135041_rms_spl3.mat,2015-11-12 13:50:40.999998,pa0001au_002_151112_135041,0001,2015-11-12,-23.824642,-42.495492,20.074000
pa0001au_003_151112_135241_rms_spl3.mat,2015-11-12 13:52:41.000001,pa0001au_003_151112_135241,0001,2015-11-12,-23.824228,-42.495501,37.126000
pa0001au_004_151112_135441_rms_spl3.mat,2015-11-12 13:54:41.000004,pa0001au_004_151112_135441,0001,2015-11-12,-23.823816,-42.495506,50.867000
pa0002au_001_151112_143618_rms_spl3.mat,2015-11-12 14:36:17.999998,pa0002au_001_151112_143618,0002,2015-11-12,-23.817508,-42.495013,5.413571
...,...,...,...,...,...,...,...
pa0296au_008_211104_061338_rms_spl3.mat,2021-11-04 06:13:37.999999,pa0296au_008_211104_061338,0296,2021-11-04,-24.458063,-42.470370,586.134284
pa0296au_009_211104_062338_rms_spl3.mat,2021-11-04 06:23:38.000003,pa0296au_009_211104_062338,0296,2021-11-04,-24.458968,-42.470625,664.644524
pa0296au_010_211104_063338_rms_spl3.mat,2021-11-04 06:33:37.999998,pa0296au_010_211104_063338,0296,2021-11-04,-24.459786,-42.470928,741.901279


In [268]:
def nettab_name_csv(line):
    '''
    Getting the station name code from other dataframe
    '''
    
    filtro = (line.date >= ciclos_dataframe_csvS["starttime"]) & (line.date <= ciclos_dataframe_csvS["endtime"])
    
    df2 = ciclos_dataframe_csvS[filtro]

    if len(df2) > 0:
    #find the nearest date and append the 2-hexatrigesimal-digit:
        return df2['station_nettab'].values[0]

In [269]:
df_csv_meanS['station_nettab'] = df_csv_meanS.apply(nettab_name_csv,axis=1)

In [270]:
df_csv_meanS

,time,filename_mseed,mergulho,date,lat,lon,dep,station_nettab
filename,,,,,,,,
pa0001au_001_151112_134841_rms_spl3.mat,2015-11-12 13:48:40.999995,pa0001au_001_151112_134841,0001,2015-11-12,-23.825054,-42.495485,4.364921,G1500
pa0001au_002_151112_135041_rms_spl3.mat,2015-11-12 13:50:40.999998,pa0001au_002_151112_135041,0001,2015-11-12,-23.824642,-42.495492,20.074000,G1500
pa0001au_003_151112_135241_rms_spl3.mat,2015-11-12 13:52:41.000001,pa0001au_003_151112_135241,0001,2015-11-12,-23.824228,-42.495501,37.126000,G1500
pa0001au_004_151112_135441_rms_spl3.mat,2015-11-12 13:54:41.000004,pa0001au_004_151112_135441,0001,2015-11-12,-23.823816,-42.495506,50.867000,G1500
pa0002au_001_151112_143618_rms_spl3.mat,2015-11-12 14:36:17.999998,pa0002au_001_151112_143618,0002,2015-11-12,-23.817508,-42.495013,5.413571,G1500
...,...,...,...,...,...,...,...,...
pa0296au_008_211104_061338_rms_spl3.mat,2021-11-04 06:13:37.999999,pa0296au_008_211104_061338,0296,2021-11-04,-24.458063,-42.470370,586.134284,G211E
pa0296au_009_211104_062338_rms_spl3.mat,2021-11-04 06:23:38.000003,pa0296au_009_211104_062338,0296,2021-11-04,-24.458968,-42.470625,664.644524,G211E
pa0296au_010_211104_063338_rms_spl3.mat,2021-11-04 06:33:37.999998,pa0296au_010_211104_063338,0296,2021-11-04,-24.459786,-42.470928,741.901279,G211E


In [271]:
df_csv_meanS['date_start_nettab'] = df_csv_meanS.time.dt.strftime('%Y/%j:%H%M')

In [272]:
df_csv_meanS

,time,filename_mseed,mergulho,date,lat,lon,dep,station_nettab,date_start_nettab
filename,,,,,,,,,
pa0001au_001_151112_134841_rms_spl3.mat,2015-11-12 13:48:40.999995,pa0001au_001_151112_134841,0001,2015-11-12,-23.825054,-42.495485,4.364921,G1500,2015/316:1348
pa0001au_002_151112_135041_rms_spl3.mat,2015-11-12 13:50:40.999998,pa0001au_002_151112_135041,0001,2015-11-12,-23.824642,-42.495492,20.074000,G1500,2015/316:1350
pa0001au_003_151112_135241_rms_spl3.mat,2015-11-12 13:52:41.000001,pa0001au_003_151112_135241,0001,2015-11-12,-23.824228,-42.495501,37.126000,G1500,2015/316:1352
pa0001au_004_151112_135441_rms_spl3.mat,2015-11-12 13:54:41.000004,pa0001au_004_151112_135441,0001,2015-11-12,-23.823816,-42.495506,50.867000,G1500,2015/316:1354
pa0002au_001_151112_143618_rms_spl3.mat,2015-11-12 14:36:17.999998,pa0002au_001_151112_143618,0002,2015-11-12,-23.817508,-42.495013,5.413571,G1500,2015/316:1436
...,...,...,...,...,...,...,...,...,...
pa0296au_008_211104_061338_rms_spl3.mat,2021-11-04 06:13:37.999999,pa0296au_008_211104_061338,0296,2021-11-04,-24.458063,-42.470370,586.134284,G211E,2021/308:0613
pa0296au_009_211104_062338_rms_spl3.mat,2021-11-04 06:23:38.000003,pa0296au_009_211104_062338,0296,2021-11-04,-24.458968,-42.470625,664.644524,G211E,2021/308:0623
pa0296au_010_211104_063338_rms_spl3.mat,2021-11-04 06:33:37.999998,pa0296au_010_211104_063338,0296,2021-11-04,-24.459786,-42.470928,741.901279,G211E,2021/308:0633


In [273]:
df_campanha = df_csv_meanS.groupby(["station_nettab"]).agg(pd.Series.tolist)

In [274]:
df_campanha

,time,filename_mseed,mergulho,date,lat,lon,dep,date_start_nettab
station_nettab,,,,,,,,
G1500,"[2015-11-12 13:48:40.999995, 2015-11-12 13:50:...","[pa0001au_001_151112_134841, pa0001au_002_1511...","[0001, 0001, 0001, 0001, 0002, 0002, 0002, 000...","[2015-11-12, 2015-11-12, 2015-11-12, 2015-11-1...","[-23.825053500000003, -23.8246425, -23.8242279...","[-42.495485, -42.4954925, -42.495501000000004,...","[4.36492065, 20.073999999999998, 37.1260000000...","[2015/316:1348, 2015/316:1350, 2015/316:1352, ..."
G1601,"[2016-01-08 10:14:23.000003, 2016-01-08 10:16:...","[pa0001au_001_160108_101423, pa0001au_002_1601...","[0001, 0001, 0001, 0001, 0002, 0002, 0002, 000...","[2016-01-08, 2016-01-08, 2016-01-08, 2016-01-0...","[-23.8396465, -23.8402455, -23.840843999999997...","[-42.50416, -42.504215, -42.5042725, -42.50433...","[4.4947145, 20.937999499999997, 37.24699850000...","[2016/008:1014, 2016/008:1016, 2016/008:1018, ..."
G1602,"[2016-02-03 00:00:57.000003, 2016-02-03 00:02:...","[pa0007au_001_160203_000057, pa0007au_002_1602...","[0007, 0007, 0007, 0007, 0007, 0007, 0007, 000...","[2016-02-03, 2016-02-03, 2016-02-03, 2016-02-0...","[-24.3285225, -24.3279755, -24.3273995, -24.32...","[-42.323730499999996, -42.323891, -42.324061, ...","[2.3261719000000003, 22.924609, 42.826563, 60....","[2016/034:0000, 2016/034:0002, 2016/034:0004, ..."
G1604,"[2016-07-08 17:20:07.999999, 2016-07-08 17:23:...","[160708_172008_pa0003au_001, 160708_172307_pa0...","[0003, 0003, 0003, 0003, 0003, 0003, 0003, 000...","[2016-07-08, 2016-07-08, 2016-07-08, 2016-07-0...","[-24.391225000000002, -24.391524333333333, -24...","[-43.821711, -43.82178233333334, -43.821897, -...","[7.286890366666666, 20.01364633333333, 40.3452...","[2016/190:1720, 2016/190:1723, 2016/190:1726, ..."
G1605,"[2016-08-17 07:45:40.999996, 2016-08-17 07:48:...","[160817_074541_pa0005au_001, 160817_074841_pa0...","[0005, 0005, 0005, 0005, 0005, 0005, 0005, 000...","[2016-08-17, 2016-08-17, 2016-08-17, 2016-08-1...","[-24.812242333333334, -24.812640666666667, -24...","[-43.140601333333336, -43.14077266666666, -43....","[7.344921800000001, 29.604296333333334, 55.430...","[2016/230:0745, 2016/230:0748, 2016/230:0751, ..."
G1606,"[2016-09-16 23:59:00.000004, 2016-09-17 00:01:...","[160916_235900_pa0003au_061, 160917_000159_pa0...","[0003, 0003, 0003, 0005, 0005, 0005, 0005, 000...","[2016-09-17, 2016-09-17, 2016-09-17, 2016-09-1...","[-24.36140633333333, -24.361825999999997, -24....","[-43.14263166666667, -43.14266466666666, -43.1...","[929.5903166666667, 945.9947333333333, 953.780...","[2016/260:2359, 2016/261:0001, 2016/261:0004, ..."
G1607,"[2016-10-21 09:07:15.000004, 2016-10-21 09:10:...","[161021_090715_pa0001au_001, 161021_091015_pa0...","[0001, 0001, 0001, 0001, 0008, 0008, 0008, 000...","[2016-10-21, 2016-10-21, 2016-10-21, 2016-10-2...","[-24.669154000000002, -24.66936733333333, -24....","[-42.88656366666667, -42.88725066666667, -42.8...","[2.3863281333333335, 13.645859333333334, 38.87...","[2016/295:0907, 2016/295:0910, 2016/295:0913, ..."
G1608,"[2016-11-21 04:11:53.999996, 2016-11-21 04:14:...","[161121_041154_pa0004au_001, 161121_041454_pa0...","[0004, 0004, 0004, 0004, 0004, 0004, 0004, 000...","[2016-11-21, 2016-11-21, 2016-11-21, 2016-11-2...","[-25.762361999999996, -25.762407333333332, -25...","[-44.433258, -44.433386000000006, -44.43356500...","[10.6926036, 27.254529333333334, 49.6655986666...","[2016/326:0411, 2016/326:0414, 2016/326:0417, ..."
G1709,"[2017-01-12 14:54:11.000002, 2017-01-12 14:57:...","[170112_145411_pa0225au_001, 170112_145710_pa0...","[0225, 0225, 0225, 0225, 0225, 0225, 0225, 022...","[2017-01-12, 2017-01-12, 2017-01-12, 2017-01-1...","[-26.011415, -26.011335666666668, -26.011274, ...","[-44.25060133333333, -44.25058066666667, -44.2...","[12.373515133333333, 32.486198333333334, 45.72...","[2017/012:1454, 2017/012:1457, 2017/012:1500, ..."


In [ ]:
df_mergulho_feather

In [ ]:
GL_info = pd.DataFrame.from_dict(glider_information_dic)
GL_info['start'] = pd.to_datetime(GL_info.start, format='%d/%m/%Y')
GL_info['end'] = pd.to_datetime(GL_info.end, format='%d/%m/%Y')

GL_info['start'] = GL_info['start'].dt.date 
GL_info['end'] = GL_info['end'].dt.date 

In [ ]:
GL_info

In [ ]:
GL_info.plot('name','start','scatter',c='k',ylabel='Data',xlabel='Modelo')

In [ ]:
for i in df_mergulho_feather.iterrows():
    # Station lines (one line per station/sensor/epoch) # code description datalogger%sn seismometer%sn # sampling orientation lat. lon. elev. depth. start end
    #Sl: UNAP "Uni-Iquique/Chile"   DM24%A1383 CMG-3ESP/60%T34622 100 ZNE -20.24393 -70.14041 0.0    0.0 2009/134 
    
    # Select DataFrame rows between two dates
    
    filtro = (i[1].date >= GL_info["start"]) & (i[1].date < GL_info["end"])

    # Using pandas.DataFrame.loc to Filter Rows by Dates
    
    df2 = GL_info[filtro]
   
    print(df2)

In [ ]:
for i in df_mergulho_feather.iterrows():
    # Station lines (one line per station/sensor/epoch) # code description datalogger%sn seismometer%sn # sampling orientation lat. lon. elev. depth. start end
    #Sl: UNAP "Uni-Iquique/Chile"   DM24%A1383 CMG-3ESP/60%T34622 100 ZNE -20.24393 -70.14041 0.0    0.0 2009/134 
    
    # Select DataFrame rows between two dates
    
    filtro = (i[1].date > GL_info["start"]) & (i[1].date < GL_info["end"])

    # Using pandas.DataFrame.loc to Filter Rows by Dates
    
    df2 = GL_info[filtro]
   
    print('Sl: '+i[1].station_nettab+' Glider:'+df2['name'].values[0]+' SENSOR'+' '+'REGISTER'+' 100 Z '+str(i[1].lat)+' '+str(i[1].lon)+' 0 0 '+i[1].date_nettab)

In [ ]:
df_mergulho_feather.to_feather('/home/dIOGOLOC/Documents/df_mergulho.feather')

# Plotando o dataframe com os mergulhos: 

In [ ]:

#########################################################################################################################################################
#Figure 

# set up the plot and create a GeoAxes:
proj = ccrs.PlateCarree()

fig, ax = plt.subplots(1,1,figsize=(16,16))
ax = plt.subplot(1, 1, 1, projection=proj)
                    
# ----------------------------------------------------------------------------------------------------------
# Limit the extent of the map to a small longitude/latitude range.
latmin=-30
latmax=-20
lonmin=-50
lonmax=-40

ax.set_extent([lonmin,lonmax, latmin, latmax], crs=ccrs.Geodetic())

# ----------------------------------------------------------------------------------------------------------
# Ploting lat/lon values
                
h = ax.scatter(df_mergulho['lon'].values,df_mergulho['lat'].values,c=np.array([mdates.date2num(i) for i in df_mergulho['time'].values]),marker='o',alpha=0.8,cmap='plasma',s=75,transform=proj)

# ----------------------------------------------------------------------------------------------------------
# Adding background map 
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE,linewidth=0.3)
ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
    
# ----------------------------------------------------------------------------------------------------------
# Adding colorbar
divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="1%", pad=0.6, axes_class=plt.Axes)

fig.add_axes(ax_cb)
cb = plt.colorbar(h, cax=ax_cb)
cb.ax.yaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y'))
   
# ----------------------------------------------------------------------------------------------------------
# Adding global location map
# inset location relative to main plot (ax) in normalized units
inset_x = 0
inset_y = 1
inset_size = 0.25

# Adding Geoaxes
ax2 = plt.axes([0, 0, 1, 1], projection=ccrs.Orthographic(central_latitude=(latmin + latmax)/2,central_longitude=(lonmin + lonmax) / 2))
ax2.set_global()

# Adding background map 
ax2.add_feature(cfeature.LAND)
ax2.add_feature(cfeature.OCEAN)
ax2.add_feature(cfeature.COASTLINE)

# Adding inset geoaxes position
ip = InsetPosition(ax, [inset_x - inset_size / 2,
                          inset_y - inset_size / 2,
                          inset_size,inset_size])
    
ax2.set_axes_locator(ip)


# Adding red rectangle position
nvert = 100
lons = np.r_[np.linspace(lonmin, lonmin, nvert),
                                 np.linspace(lonmin, lonmax, nvert),
                                 np.linspace(lonmax, lonmax, nvert)].tolist()
    
lats = np.r_[np.linspace(latmin, latmax, nvert),
                                 np.linspace(latmax, latmax, nvert),
                                 np.linspace(latmax, latmin, nvert)].tolist()

ring = LinearRing(list(zip(lons, lats)))
ax2.add_geometries([ring], ccrs.PlateCarree(),facecolor='none', edgecolor='red', linewidth=0.75)

        
# ----------------------------------------------------------------------------------------------------------
# Saving figure
#os.makedirs(FOLDER_OUTPUT+'FIGURAS/INTERP_MSEED/'+st[0].stats.starttime.strftime('%Y')+'/'+st[0].stats.starttime.strftime('%Y-%m-%d')+'/',exist_ok=True)
#fig.savefig(FOLDER_OUTPUT+'FIGURAS/INTERP_MSEED/'+st[0].stats.starttime.strftime('%Y')+'/'+st[0].stats.starttime.strftime('%Y-%m-%d')+'/'+'interp_mseed_'+file_n_meta+'.png')
#plt.close()

In [ ]:
# set up plotting
fig = plt.figure()
ax = plt.axes()

h = ax.scatter(df_mergulho['lon'].values,df_mergulho['lat'].values,c=np.array([mdates.date2num(i) for i in df_mergulho['time'].values]),marker='o',alpha=0.8,cmap='plasma',s=75,transform=proj)


ax.plot(ttime,tdata,c='k',lw=0.1,alpha=0.3)
line, = ax.plot([],[],c='r', lw=0.5,alpha=0.7)
ax.set_xlim(ttime[0],ttime[-1])
ax.set_ylim(tdata.min(), tdata.max())

ax.set_yticks([])

# methods for animation
def init():
    line.set_data([],[])
    return line,

def animate(i):
    y = wave_to_plot[i]
    x = time_to_plot[i]
    line.set_data(x,y)

    return line,

# Save the animation
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=len(wave_to_plot))
HTML(anim.to_jshtml())

# Save the animation as an 
f = r"/home/diogoloc/dados_posdoc/ON02_analysis/Figuras/Localizacao_navio_ponte_ON_ON02.mp4" 
writervideo = animation.FFMpegWriter(fps=10) 
anim.save(f, writer=writervideo)
